- problem definition: 
Task 1: Minimise the overall delivery time and cost of delivery.
Task 2: Maximise cost-efficiency (total gas delivered/overall cost of delivery)

- Constraints:
Task 1: Deliver to customers with less gas than 50% of their tank's capacity.
Task 2: Deliver to customers with less gas than 80% of their tank's capacity and fill up to 80% of their tank's capacity.

- optimization algorithm (+ others): 
Task 1: Greedy search algorithm, Breadth-First-Search algorithm
Task 2: Continuous Genetic Algorithm, Breadth-First-Search algorithm

- results:
outputted to task1_solution.json file 

- methodology:

1. Converting the .csv files into pandas dataframe (using pandas library)

Firstly I converted the data stored in .csv files into a pandas dataframe. There are many advantages to this (1), as well as some disadvantages (2): 
+ Python supported operations 
+ NumPy supported operations 
+ Extensive set of feature techniques 
+ Readable syntax 
+ Cleaner representation of data structures 
+ Conversion of .csv to a tabular format
- Documenation (less coverage on the harder functions of pandas library)

In [3]:
# EXAMPLE CODE FOR .CSV FILE CONVERSION INTO PANDAS 

# relevant imports 
import pandas as pd 

# location dataframe
location_df = pd.read_csv('SaO_Optilandia_resub_locations.csv')

# visual representation of location_df in tabular format
location_df.head()

,id,x,y,is_depot,is_customer,capacity,level
0,0,49480.9323,5936.91874,False,False,NaN,NaN
1,1,49707.2278,6326.20930,False,False,NaN,NaN
2,2,49535.7448,6405.08264,False,False,NaN,NaN
3,3,49755.2884,6117.62951,False,False,NaN,NaN
4,4,49781.7988,5722.16266,False,False,NaN,NaN


2. Implementing the use of numPy library to process data series from a pandas dataframe

NumPy library deemed important for processing and operating on large data structures stored in the relative dataframes. The advantages and disadvantages of using NumPy library include (3): 

+ As opposed to the tabular data representation in Pandas used mainly for analysing data, NumPy offers multi-dimensional arrays and matrix datastructures for performing mathematical operations. 


- References: 

(1) https://peerj.com/articles/cs-55.pdf 
(2) https://data-flair.training/blogs/advantages-of-python-pandas/ 
(3) https://numpy.org/devdocs/user/absolute_beginners.html#:~:text=NumPy%20arrays%20are%20faster%20and,to%20be%20optimized%20even%20further. 
(4) https://www.javatpoint.com/pandas-vs-numpy 
(5) https://medium.com/fintechexplained/why-should-we-use-numpy-c14a4fb03ee9#:~:text=NumPy%20is%20an%20open%2Dsource,%2C%20statistical%2C%20and%20algebraic%20routines. 
